In [ ]:
!pip install ultralytics


In [ ]:
import os
from ultralytics import YOLO
from google.colab import drive
import matplotlib.pyplot as plt
import cv2


In [ ]:
drive.mount('/content/drive')


In [ ]:
DATASET_ROOT = "/content/drive/MyDrive/seg_dataset"

yaml_path = os.path.join(DATASET_ROOT, "data.yaml")
train_path = os.path.join(DATASET_ROOT, "images/train")
val_path = os.path.join(DATASET_ROOT, "images/val")
test_path = os.path.join(DATASET_ROOT, "images/test")

print("YAML path:", yaml_path)
print("Train images:", len(os.listdir(train_path)))
print("Val images:", len(os.listdir(val_path)))
print("Test images:", len(os.listdir(test_path)))


In [ ]:
det_model = YOLO("yolov8n.pt")


In [ ]:
def train_model(model, yaml_file, epochs=30, img_size=640, batch_size=16):
    print("Starting training...")
    results = model.train(
        data=yaml_file,
        epochs=epochs,
        imgsz=img_size,
        batch=batch_size,
        project="/content/drive/MyDrive/yolo_training",
        name="detection_exp1"
    )
    print("Training complete!")
    return results


In [ ]:
train_results = train_model(det_model, yaml_path, epochs=25, img_size=640, batch_size=8)


In [ ]:
val_metrics = det_model.val()
print(val_metrics)


In [ ]:
pred_results = det_model.predict(
    source=test_path,
    save=True,
    conf=0.4
)


In [ ]:
def show_prediction_image(image_path):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 6))
    plt.imshow(img_rgb)
    plt.axis("off")
    plt.show()

pred_dir = "/content/runs/detect/predict"
if os.path.exists(pred_dir):
    pred_files = os.listdir(pred_dir)
    if len(pred_files) > 0:
        first_result = os.path.join(pred_dir, pred_files[0])
        print("Showing:", first_result)
        show_prediction_image(first_result)
    else:
        print("No prediction files found!")
else:
    print("Prediction directory not found!")
